In [55]:
import pandas as pd
import numpy as np
from datetime import datetime, date
from sweref99 import projections
from datetime import datetime, date

In [56]:
fire_risk_path='./data/Brandriskdata 2000-2020.csv'
reported_fires_path='./data/Insatser till brand i skog och mark 2000-2020.xlsx'
#'PunktID': str, 'E': str, 'N': str,
rtypes={ 'PunktID': str}

In [57]:
#Reading csv file from path with dtypes
reported_fire_risk = pd.read_csv(fire_risk_path, sep=';', dtype=rtypes, )
reported_fires = pd.read_excel(reported_fires_path)

In [58]:
risk_sample = reported_fire_risk
fires_sample = reported_fires

In [59]:
print(risk_sample.isnull().sum(axis = 0))
print("Total rows: ")
print(risk_sample.shape[0])

PunktID                 0
E                       2
N                       2
Kommun                  2
Datum                   2
Temp                14617
Tmedel              38049
Nederbord           23520
RH                  33459
Vindhastighet        3991
Vindriktning     12310774
FFMC              1701172
DMC               1701172
DC                1701172
ISI               1701172
BUI               1701172
FWI               1701172
FWI_index         1701172
HBV_o             1404342
HBV_u             1404342
HBV               1404342
HBV_index         1404342
Gras              6855866
dtype: int64
Total rows: 
19188859


In [60]:
risk_sample[['E','N']] = risk_sample[['E','N']].fillna(0)
risk_sample['N'] = risk_sample['N'].astype(int)
risk_sample['E'] = risk_sample['E'].astype(int)

In [61]:
risk_sample.dropna(subset=['FWI'], how='all', inplace=True)
risk_sample.dropna(subset=['Tmedel'], how='all', inplace=True)


In [62]:
print(risk_sample.isnull().sum(axis = 0))
print("Total rows: ")
print(risk_sample.shape[0])

PunktID                 0
E                       0
N                       0
Kommun                  0
Datum                   0
Temp                    0
Tmedel                  0
Nederbord               0
RH                   3989
Vindhastighet           0
Vindriktning     11068726
FFMC                    0
DMC                     0
DC                      0
ISI                     0
BUI                     0
FWI                     0
FWI_index               0
HBV_o               39890
HBV_u               39890
HBV                 39890
HBV_index           39890
Gras              6777056
dtype: int64
Total rows: 
17474853


In [63]:
fireRisk = risk_sample.drop(columns=['Nederbord',
       'RH', 'Vindhastighet', 'Vindriktning', 'FFMC', 'DMC', 'DC', 'ISI',
       'BUI', 'HBV_o', 'HBV_u', 'HBV', 'HBV_index',
       'Gras'])
fireRisk

,PunktID,E,N,Kommun,Datum,Temp,Tmedel,FWI,FWI_index
0,1,454334,6690301,2021.0,2000-04-01,"1,8","1,6",0,1.0
1,1,454334,6690301,2021.0,2000-04-02,"-2,3","-2,3",0,1.0
2,1,454334,6690301,2021.0,2000-04-03,"-1,4",-3,0,1.0
3,1,454334,6690301,2021.0,2000-04-04,"0,1","-0,2",0,1.0
4,1,454334,6690301,2021.0,2000-04-05,"0,4","-2,8",0,1.0
...,...,...,...,...,...,...,...,...,...
19188852,7892,375482,6733336,1737.0,2020-12-27,"0,4",0,0,-1.0
19188853,7892,375482,6733336,1737.0,2020-12-28,"-0,1",0,0,-1.0
19188854,7892,375482,6733336,1737.0,2020-12-29,"0,3",0,0,-1.0
19188855,7892,375482,6733336,1737.0,2020-12-30,"-0,1",0,0,-1.0


In [64]:
#Values needs to be float to be able to do means, to convert to float we first change all ',' to '.'.
fireRisk['FWI'] = fireRisk['FWI'].str.replace(',','.')
fireRisk['Temp'] = fireRisk['Temp'].str.replace(',','.')
fireRisk['Tmedel'] = fireRisk['Tmedel'].str.replace(',','.')
fireRisk['FWI'] = fireRisk['FWI'].astype(float)
fireRisk['Temp'] = fireRisk['Temp'].astype(float)
fireRisk['Tmedel'] = fireRisk['Tmedel'].astype(float)
fireRisk['Kommun'] = fireRisk['Kommun'].astype(int)
#Datum needs to be in datetime format, here we use pd.to_datetime to convert the strings
fireRisk['Datum'] = pd.to_datetime(fireRisk['Datum'], format='%Y-%m-%d')
#For this we dont need the sweref99 coordinates and we drop these columns.
fireRiskT = fireRisk.drop(columns=['E','N'])

In [65]:
fireRiskT = fireRiskT[fireRiskT.FWI_index != -1]
fireRiskT

,PunktID,Kommun,Datum,Temp,Tmedel,FWI,FWI_index
0,1,2021,2000-04-01,1.8,1.6,0.0,1.0
1,1,2021,2000-04-02,-2.3,-2.3,0.0,1.0
2,1,2021,2000-04-03,-1.4,-3.0,0.0,1.0
3,1,2021,2000-04-04,0.1,-0.2,0.0,1.0
4,1,2021,2000-04-05,0.4,-2.8,0.0,1.0
...,...,...,...,...,...,...,...
19188791,7892,1737,2020-10-27,7.1,7.0,0.0,1.0
19188792,7892,1737,2020-10-28,8.0,7.0,0.0,1.0
19188793,7892,1737,2020-10-29,8.1,6.0,0.0,1.0
19188794,7892,1737,2020-10-30,2.9,1.0,0.0,1.0


In [66]:
#Grouping by Kommun and date (YYYY-MM-DD) and converting rest of values to mean values.
test = fireRiskT.groupby(['Datum','Kommun']).median()
test = test.reset_index()
test = test.rename(columns={"Datum": "datum", "Kommun":"kommun"})

In [67]:

#Actual merge happens here
merged = pd.merge(fires_sample, test, how='inner', on=['kommun','datum'])
#merged = merged.drop(columns='index')



In [68]:
#The code here was to let us see how many years were missing values for the winter months, and to help us decide what to do with these months
merged['month'] = merged['datum'].dt.month
#winterseason = merged[merged['month' == 11, 'month' == 12, 'month' == 1 , 'month' == 2]]
months = [10, 11, 12, 1]
winterseason = merged[merged['month'].isin(months)]
winterseason.sort_values(by= 'datum', ascending = False)

,ar,datum,tid,kommun,kommunKortNamn,verksamhetText,sweref99Norr,sweref99Ost,BEJBbrandorsakText,arealProduktivSkogsmark_m2,arealAnnanTradbevuxenMark_m2,arealMarkUtanTrad_m2,Temp,Tmedel,FWI,FWI_index,month
85204,2020,2020-10-30,18:09:00,1480,Göteborg,Verksamhet inte knuten till en byggnad,6401692.0,304234.0,Avsiktlig brand,0,0,5,9.25,7.5,0.05,1.0,10
85203,2020,2020-10-30,17:02:00,1421,Orust,Verksamhet inte knuten till en byggnad,6444616.0,294572.0,Annan eldning,0,0,2,8.90,6.0,0.20,1.0,10
85202,2020,2020-10-30,14:26:00,120,Värmdö,Verksamhet inte knuten till en byggnad,6577748.0,709755.0,Okänd,0,0,10,8.40,7.0,0.25,1.0,10
85201,2020,2020-10-29,03:29:00,2460,Vännäs,Verksamhet inte knuten till en byggnad,7096296.0,733637.0,Avsiktlig brand,0,0,1,7.75,6.0,0.00,1.0,10
85200,2020,2020-10-28,17:25:00,2514,Kalix,Verksamhet inte knuten till en byggnad,7326705.0,873357.0,Avsiktlig brand,20,0,0,5.60,4.0,0.00,1.0,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59317,2015,2015-10-02,07:49:00,180,Stockholm,Verksamhet inte knuten till en byggnad,6580733.0,674044.0,Avsiktlig brand,0,0,5,17.85,14.0,13.40,3.0,10
59321,2015,2015-10-02,16:07:00,780,Växjö,Verksamhet inte knuten till en byggnad,6305758.0,487570.0,Avsiktlig brand,0,1,0,16.55,13.0,8.55,3.0,10
59322,2015,2015-10-02,18:57:00,481,Oxelösund,Verksamhet inte knuten till en byggnad,6507365.0,617766.0,Annan,0,0,1,17.90,14.0,11.00,3.0,10
59323,2015,2015-10-02,23:59:00,126,Huddinge,Verksamhet inte knuten till en byggnad,6569672.0,679733.0,Okänd,0,1,0,18.60,14.0,13.20,3.0,10


In [69]:
#Method to replace all -1 values in FWI-index with NaN

#merged['FWI_index'] = merged['FWI_index'].replace([-1.0],np.nan)


In [76]:
#It might be interesting to look at the days where the fires of the municipalities FWI_index is above 4
above4 = merged[merged['FWI_index']>=4]
above4

,ar,datum,tid,kommun,kommunKortNamn,verksamhetText,sweref99Norr,sweref99Ost,BEJBbrandorsakText,arealProduktivSkogsmark_m2,arealAnnanTradbevuxenMark_m2,arealMarkUtanTrad_m2,Temp,Tmedel,FWI,FWI_index,month
885,2000,2000-04-29,15:12:00,862,Emmaboda,Verksamhet inte knuten till en byggnad,6280691.0,530856.0,Annan,0,0,9,26.60,17.7,17.90,4.0,4
1057,2000,2000-05-01,15:22:00,765,Älmhult,Verksamhet inte knuten till en byggnad,6257447.0,424171.0,Okänd,0,5,0,17.00,14.4,17.10,4.0,5
1090,2000,2000-05-02,11:57:00,1282,Landskrona,Verksamhet inte knuten till en byggnad,6199165.0,362917.0,Okänd,0,1,0,16.50,11.9,20.20,4.0,5
1091,2000,2000-05-02,17:10:00,1282,Landskrona,Verksamhet inte knuten till en byggnad,NaN,NaN,Okänd,0,250,0,16.50,11.9,20.20,4.0,5
1109,2000,2000-05-02,16:42:00,1283,Helsingborg,Verksamhet inte knuten till en byggnad,6213026.0,357382.0,Okänd,0,0,20,16.95,12.2,20.25,4.0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84765,2020,2020-09-02,07:07:00,1480,Göteborg,Verksamhet inte knuten till en byggnad,6391327.0,316393.0,Okänd,0,60,0,19.60,16.0,18.75,4.0,9
84766,2020,2020-09-02,19:19:00,1480,Göteborg,Verksamhet inte knuten till en byggnad,6401603.0,307402.0,Okänd,0,4,0,19.60,16.0,18.75,4.0,9
84939,2020,2020-09-17,15:25:00,1291,Simrishamn,Verksamhet inte knuten till en byggnad,6174618.0,448402.0,Okänd,600,0,0,16.35,12.5,18.10,4.0,9
85008,2020,2020-09-21,21:30:00,2183,Bollnäs,Verksamhet inte knuten till en byggnad,6797391.0,580626.0,Avsiktlig brand,750,0,0,15.90,11.0,17.20,4.0,9


In [78]:
#Removes all rows with the specified values in the specified column
def filter_rows_by_values(df, col, values):
    return df[~df[col].isin(values)]

In [79]:
#Filter cause to only include campfire and BBQ
above4 = filter_rows_by_values(above4, 'BEJBbrandorsakText', ['Fyrverkeri eller pyroteknik', 'Annan',
       'Barns lek med eld', 'Annan eldning', 'Avsiktlig brand',
       'Övriga gnistor',
       'Återantändning av brand från tidigare räddningsinsats',
       'Självantändning - biologisk eller kemisk', 'Blixtnedslag', 'Heta arbeten',
       'Rökning', 'Tågbromsning', 'Fel i utrustning', 'Uppgift saknas'])
above4

,ar,datum,tid,kommun,kommunKortNamn,verksamhetText,sweref99Norr,sweref99Ost,BEJBbrandorsakText,arealProduktivSkogsmark_m2,arealAnnanTradbevuxenMark_m2,arealMarkUtanTrad_m2,Temp,Tmedel,FWI,FWI_index,month
1057,2000,2000-05-01,15:22:00,765,Älmhult,Verksamhet inte knuten till en byggnad,6257447.0,424171.0,Okänd,0,5,0,17.00,14.4,17.10,4.0,5
1090,2000,2000-05-02,11:57:00,1282,Landskrona,Verksamhet inte knuten till en byggnad,6199165.0,362917.0,Okänd,0,1,0,16.50,11.9,20.20,4.0,5
1091,2000,2000-05-02,17:10:00,1282,Landskrona,Verksamhet inte knuten till en byggnad,NaN,NaN,Okänd,0,250,0,16.50,11.9,20.20,4.0,5
1109,2000,2000-05-02,16:42:00,1283,Helsingborg,Verksamhet inte knuten till en byggnad,6213026.0,357382.0,Okänd,0,0,20,16.95,12.2,20.25,4.0,5
1382,2000,2000-05-07,02:26:00,180,Stockholm,Verksamhet inte knuten till en byggnad,NaN,NaN,Okänd,0,20,0,16.90,10.0,19.65,4.0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84706,2020,2020-08-28,14:57:00,1980,Västerås,Verksamhet inte knuten till en byggnad,6609656.0,589536.0,Okänd,0,0,3,18.10,12.0,25.10,5.0,8
84707,2020,2020-08-28,15:22:00,1885,Lindesberg,Verksamhet inte knuten till en byggnad,6612185.0,502645.0,Okänd,0,1,0,17.30,12.0,20.95,4.0,8
84765,2020,2020-09-02,07:07:00,1480,Göteborg,Verksamhet inte knuten till en byggnad,6391327.0,316393.0,Okänd,0,60,0,19.60,16.0,18.75,4.0,9
84766,2020,2020-09-02,19:19:00,1480,Göteborg,Verksamhet inte knuten till en byggnad,6401603.0,307402.0,Okänd,0,4,0,19.60,16.0,18.75,4.0,9


In [80]:
above4['TotArea'] = above4['arealProduktivSkogsmark_m2']+above4['arealAnnanTradbevuxenMark_m2']+above4['arealMarkUtanTrad_m2']
above4['Hectares'] = above4['TotArea']/10000

C:\Users\Joel\AppData\Local\Temp/ipykernel_9996/2346100680.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  above4['TotArea'] = above4['arealProduktivSkogsmark_m2']+above4['arealAnnanTradbevuxenMark_m2']+above4['arealMarkUtanTrad_m2']
C:\Users\Joel\AppData\Local\Temp/ipykernel_9996/2346100680.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  above4['Hectares'] = above4['TotArea']/10000


In [94]:
#Creates two tables. One is between 2000 and 2018 (before disposable grill ban), and the second is 2019-2020 (after disposable grill ban)
tablesplit = above4.set_index(['datum'])
pre2019 = tablesplit.loc['2000-1-1':'2018-12-31']
post2018 = tablesplit.loc['2019-1-1' : '2020-12-31']
post2018

,ar,tid,kommun,kommunKortNamn,verksamhetText,sweref99Norr,sweref99Ost,BEJBbrandorsakText,arealProduktivSkogsmark_m2,arealAnnanTradbevuxenMark_m2,arealMarkUtanTrad_m2,Temp,Tmedel,FWI,FWI_index,month,TotArea,Hectares
datum,,,,,,,,,,,,,,,,,,
2019-04-02,2019,14:26:00,563,Valdemarsvik,Verksamhet inte knuten till en byggnad,6460120.0,613068.0,Okänd,0,35000,0,8.90,2.0,17.30,4.0,4,35000,3.5000
2019-04-02,2019,15:14:00,180,Stockholm,Verksamhet inte knuten till en byggnad,6577075.0,668257.0,Okänd,0,10,0,9.15,5.0,18.40,4.0,4,10,0.0010
2019-04-02,2019,19:03:00,191,Sigtuna,Verksamhet inte knuten till en byggnad,6612192.0,658760.0,Okänd,0,0,80,8.80,4.0,18.20,4.0,4,80,0.0080
2019-04-15,2019,19:30:00,763,Tingsryd,Verksamhet inte knuten till en byggnad,6267940.0,497162.0,Okänd,0,0,5,10.30,3.0,17.15,4.0,4,5,0.0005
2019-04-16,2019,11:43:00,1081,Ronneby,Verksamhet inte knuten till en byggnad,6238344.0,511887.0,Grillning eller lägereld,0,2400,0,10.95,5.0,20.10,4.0,4,2400,0.2400
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-08-28,2020,14:57:00,1980,Västerås,Verksamhet inte knuten till en byggnad,6609656.0,589536.0,Okänd,0,0,3,18.10,12.0,25.10,5.0,8,3,0.0003
2020-08-28,2020,15:22:00,1885,Lindesberg,Verksamhet inte knuten till en byggnad,6612185.0,502645.0,Okänd,0,1,0,17.30,12.0,20.95,4.0,8,1,0.0001
2020-09-02,2020,07:07:00,1480,Göteborg,Verksamhet inte knuten till en byggnad,6391327.0,316393.0,Okänd,0,60,0,19.60,16.0,18.75,4.0,9,60,0.0060


In [95]:
#Average amount of fires per year, during "eldningförbud" (FWI-index >= 4), before and after disposable grill ban
#pre2019 has 11755 fires over 19 years. post2018 has 1547 fires over 2 years. Average for each periods:
beforeban = 11755/19
afterban = 1547/2

In [99]:
#Average amount of fires per year(during fire hazard warning "eldningsförbud") before disposable grill ban
beforeban

618.6842105263158

In [98]:
#Average amount of fires per year(during fire hazard warning "eldningsförbud") after disposable grill ban
afterban

773.5